In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import json
import numpy as np

from collections import Counter
from folium.plugins import MarkerCluster
from sklearn.cluster import DBSCAN

from utils import *
from FedRank import FedRank

In [67]:
from sklearn.metrics import hinge_loss

In [2]:
# develop mode
%load_ext autoreload
%autoreload 2

In [3]:
# Moscow coordinates
MOW = [55.75222, 37.61556]

r = 0.02
r2 = r**2

In [56]:
r2

0.0004

In [4]:
# load, clean and group train data
train_df = clean_df('./data/train_set.csv')
train_gr = train_df.groupby('customer_id')

test_df = clean_df('./data/test_set.csv')
test_gr = test_df.groupby('customer_id')

/home/frodos/raifaizen/.env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/frodos/raifaizen/.env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### MCC
Подгружаем MCC коды, вычисляем их частотность в тренировочных данных. Выбираем MMC, которые встречаются более 10 000 раз и сохраняем их в mcc_list.

In [5]:
# load mcc codes
mcc_codes = pd.read_csv('./data/mcc_codes.csv', index_col='mcc')
mcc_counts = train_df['mcc'].value_counts().to_frame(name = 'count')
mcc_codes = mcc_counts.merge(mcc_codes, left_index=True, right_index=True)
mcc_list = mcc_codes.query('count >10000').index.values

In [6]:
mcc_codes.loc[:,['count','edited_description']].head(20)

,count,edited_description
5411,391635,"Grocery Stores, Supermarkets"
6011,275228,Financial Institutions – Manual Cash Disbursem...
5814,128771,Fast Food Restaurants
5812,62407,Eating places and Restaurants
5499,44703,Misc. Food Stores – Convenience Stores and Spe...
5541,42296,Service Stations ( with or without ancillary s...
5912,41175,Drug Stores and Pharmacies
4111,24899,Local/Suburban Commuter Passenger Transportati...
5921,18353,"Package Stores – Beer, Wine, and Liquor"
5331,13042,Variety Stores


In [7]:
best_dbscan_params = {'eps': 0.036, 'min_samples': 2}
clusters = get_dbscan_clust(train_gr, best_dbscan_params, mcc_list=mcc_list)

In [8]:
train_df = train_df.merge(clusters, left_index=True, right_index=True, how='left')
train_df.clust_label.fillna(-2, inplace=True)
t_gr = train_df.groupby(['customer_id','clust_label'])

In [9]:
train_df.head(5)

,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon,clust_label
0,2.884034,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177,0
1,2.775633,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177,0
2,3.708368,St Petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177,-2
3,2.787498,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177,0
4,2.892510,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177,0


### Создаём допольнительные фичи
Проходимся по всем кластерам всех пользователей и считаем удельный вклад каждого MCC по количеству операций.

На выходе получаем таблицу, в которой колонки соответсвуют MCC

In [10]:
### Create features with relative contributions of each MCC code for each cluster of each customer
def get_mcc_features(gr):
    df = gr.mcc.aggregate('value_counts', **{'normalize':True}).unstack(level=-1).fillna(0)
    df.columns = ["mcc_%s" % (n1) for n1 in df.columns.values]
    return df


def get_amount_features(gr):
    agg_dict = {
    'amount': [np.mean, np.max, np.min, 'count', 'std', 'sum']
    }
    c = gr.aggregate(agg_dict)
    newidx = []
    for (n1,n2) in c.columns.ravel():
        newidx.append("%s_%s" % (n1,n2))
    c.columns=newidx
    
    rel_features = c.loc[:,['amount_sum','amount_count']].groupby(level=0).transform(lambda x: x/x.sum())
    rel_features.columns = [x+'_rel' for x in rel_features.columns]
    
    return c.merge(rel_features, left_index=True, right_index=True)

def get_clust_pos(gr):
    agg_dict = {
        'pos_address_lat': np.mean,
        'pos_address_lon': np.mean
        }
    return gr.aggregate(agg_dict)

In [13]:
train_features = get_amount_features(t_gr).merge(get_mcc_features(t_gr), 
                                                        left_index=True, right_index=True)

In [14]:
train_features

amount_mean  amount_amax  \
customer_id                      clust_label                             
0001f322716470bf9bfc1708f06f00fc -2              2.787178     3.143099   
                                  0              3.309715     4.614833   
                                  1              2.553616     2.973381   
                                  2              3.255826     4.481342   
0007297d86e14bd68bd87b1dbdefe302 -2              2.818547     4.696914   
                                 -1              2.351594     2.351594   
                                  0              2.491890     3.197908   
                                  1              2.792455     4.590551   
                                  2              2.820427     3.401110   
                                  3              2.367055     2.712781   
                                  4              2.951470     4.499151   
                                  5              2.488475     2.618730   
                                  6              2.159304     2.373256   
                                  7              2.910833     3.219585   
                                  8              2.843656     3.703698   
000b709c6c6fb1e8efcfd95e57c2a9de -2              3.317638     4.319009   
                                 -1              2.178760     2.707712   
                                  0              2.454432     2.539817   
                                  1              2.701774     2.713293   
                                  2              2.813341     3.521626   
                                  3              3.050185     3.475695   
                                  4              2.567378     2.578458   
                                  5              1.665275     1.678664   
                                  6              3.459745     3.482396   
                                  7              3.034875     3.906433   
0027a7618d97cc9fbda55fac457eaeb7 -1              2.102438     2.102438   
                                  0              2.719135     3.224442   
                                  1              3.215212     3.359777   
                                  2              2.944809     3.375377   
002b9f6e118c54f1292e03d1a04d516e -2              3.389237     4.089807   
...                                                   ...          ...   
ff92d5420f5fb92a37e1280d1fc9e5f4 -1              3.711420     3.711420   
                                  0              2.705312     3.104000   
                                  1              2.390978     2.498249   
                                  2              4.685161     4.687912   
ffaeae55d4dbf29058f04e7a6a764f02 -2              2.665045     3.340622   
                                 -1              2.873541     2.873541   
                                  0              3.160077     3.224281   
                                  1              3.009963     3.534924   
                                  2              2.782018     3.070825   
ffc5289194413ec68c3f7adc8121d69b -2              3.599410     3.599410   
                                 -1              2.680235     2.680235   
                                  0              2.173551     2.723632   
                                  1              2.772851     3.046552   
                                  2              2.514433     2.636701   
                                  3              3.343938     4.152719   
ffd097949a4a238296a7deadfb376cc0 -1              2.784659     2.784659   
                                  0              2.812380     3.445825   
                                  1              2.986624     3.170735   
                                  2              3.366384     3.366384   
                                  3              3.201811     3.201811   
ffdd5ec2a90e355cf40525eac1a6fd34 -2              3.324806     4.404513   
                                 -1              2

In [19]:
train_df.head()

,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon,clust_label
0,2.884034,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177,0
1,2.775633,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177,0
2,3.708368,St Petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177,-2
3,2.787498,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177,0
4,2.892510,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177,0


In [27]:
dist_dict = {
    'home_add_lat': 'median',
    'home_add_lon': 'median',
    'work_add_lat': 'median',
    'work_add_lon': 'median'
    
}

train_hw_pos = train_df.groupby(['customer_id','clust_label']).aggregate(dist_dict)
train_hw_pos = train_hw_pos.merge(get_clust_pos(t_gr), left_index=True, right_index=True)
train_hw_pos.head(10)

home_add_lat  home_add_lon  \
customer_id                      clust_label                               
0001f322716470bf9bfc1708f06f00fc -2                 44.708        37.775   
                                  0                 44.708        37.775   
                                  1                 44.708        37.775   
                                  2                 44.708        37.775   
0007297d86e14bd68bd87b1dbdefe302 -2                 55.799        37.388   
                                 -1                 55.799        37.388   
                                  0                 55.799        37.388   
                                  1                 55.799        37.388   
                                  2                 55.799        37.388   
                                  3                 55.799        37.388   

                                              work_add_lat  work_add_lon  \
customer_id                      clust_label                               
0001f322716470bf9bfc1708f06f00fc -2                 44.735        37.798   
                                  0                 44.735        37.798   
                                  1                 44.735        37.798   
                                  2                 44.735        37.798   
0007297d86e14bd68bd87b1dbdefe302 -2                    NaN           NaN   
                                 -1                    NaN           NaN   
                                  0                    NaN           NaN   
                                  1                    NaN           NaN   
                                  2                    NaN           NaN   
                                  3                    NaN           NaN   

                                              pos_address_lat  pos_address_lon  
customer_id                      clust_label                                    
0001f322716470bf9bfc1708f06f00fc -2                 44.745117        37.726500  
                                  0                 44.728065        37.794945  
                                  1                 56.251347        43.446254  
                                  2                  0.000000         0.000000  
0007297d86e14bd68bd87b1dbdefe302 -2                 56.019628        37.070111  
                                 -1                 45.102668        38.980771  
                                  0                 55.809137        37.462241  
                                  1                 55.801699        37.403000  
                                  2                 55.822604        37.524894  
                                  3                 55.675554        37.505458

In [ ]:
target_dist = get_target_distances(train_hw_pos)
target_rank = target_dist.fillna(999).groupby(level=0).transform(lambda x: (x.rank()==1).astype(int))

In [60]:
target_rank

dist_home  dist_work
customer_id                      clust_label                      
0001f322716470bf9bfc1708f06f00fc -2                   0          0
                                  0                   1          1
                                  1                   0          0
                                  2                   0          0
0007297d86e14bd68bd87b1dbdefe302 -2                   0          0
                                 -1                   0          0
                                  0                   0          0
                                  1                   1          0
                                  2                   0          0
                                  3                   0          0
                                  4                   0          0
                                  5                   0          0
                                  6                   0          0
                                  7                   0          0
                                  8                   0          0
000b709c6c6fb1e8efcfd95e57c2a9de -2                   0          0
                                 -1                   0          0
                                  0                   0          0
                                  1                   0          0
                                  2                   1          1
                                  3                   0          0
                                  4                   0          0
                                  5                   0          0
                                  6                   0          0
                                  7                   0          0
0027a7618d97cc9fbda55fac457eaeb7 -1                   0          0
                                  0                   0          0
                                  1                   0          0
                                  2                   1          0
002b9f6e118c54f1292e03d1a04d516e -2                   0          0
...                                                 ...        ...
ff92d5420f5fb92a37e1280d1fc9e5f4 -1                   0          0
                                  0                   1          1
                                  1                   0          0
                                  2                   0          0
ffaeae55d4dbf29058f04e7a6a764f02 -2                   0          0
                                 -1                   1          0
                                  0                   0          0
                                  1                   0          0
                                  2                   0          0
ffc5289194413ec68c3f7adc8121d69b -2                   0          0
                                 -1                   0          0
                                  0                   0          0
                                  1                   1          0
                                  2                   0          0
                                  3                   0          0
ffd097949a4a238296a7deadfb376cc0 -1                   0          0
                                  0                   0          1
                                  1                   1          0
                                  2                   0          0
                                  3                   0          0
ffdd5ec2a90e355cf40525eac1a6fd34 -2                   0          0
                                 -1                   0          0
                                  0                   0          0
                                  1                   1          0
                                  2                   0          1
                                  3                   0          0
                                  4                   0          0
                          

In [91]:
test_user = '0001f322716470bf9bfc1708f06f00fc'
#test_user = '0007297d86e14bd68bd87b1dbdefe302'
y_true = target_rank.loc[test_user,'dist_home']
y_pr = np.zeros(y_true.shape)
y_pr[0] = 1
print(y_true.shape, hinge_loss(y_true, y_pr) /(1-1.0/y_true.shape[0]))

(4,) 1.6666666666666667


In [106]:
softmax(np.array([[1, 0, 0, 0.1]]))

array([[0.46678181, 0.17171943, 0.17171943, 0.18977932]])

In [220]:
coeffs = np.random.rand(train_features.shape[1], 1)

In [266]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [269]:
y_pr = pd.DataFrame(np.matmul(train_features,coeffs), index= train_features.index, columns=['y'])
y_pr2 = y_pr.fillna(0).groupby(level=0).transform(lambda x: softmax(x))
y_pr2

y
customer_id                      clust_label              
0001f322716470bf9bfc1708f06f00fc -2           1.928413e-14
                                  0           1.038394e-04
                                  1           2.615784e-14
                                  2           9.998962e-01
0007297d86e14bd68bd87b1dbdefe302 -2           8.666884e-19
                                 -1           4.889978e-30
                                  0           9.122089e-27
                                  1           1.822909e-04
                                  2           8.393982e-28
                                  3           7.892106e-25
                                  4           9.998177e-01
                                  5           2.392669e-28
                                  6           1.355341e-27
                                  7           4.156258e-28
                                  8           2.790112e-26
000b709c6c6fb1e8efcfd95e57c2a9de -2           9.949515e-01
                                 -1           1.900149e-03
                                  0           1.068146e-04
                                  1           4.476657e-04
                                  2           7.636806e-04
                                  3           4.979079e-04
                                  4           1.249823e-04
                                  5           4.625071e-05
                                  6           2.599049e-04
                                  7           9.011863e-04
0027a7618d97cc9fbda55fac457eaeb7 -1           2.320674e-04
                                  0           9.189594e-01
                                  1           2.833914e-02
                                  2           5.246943e-02
002b9f6e118c54f1292e03d1a04d516e -2           4.759171e-26
...                                                    ...
ff92d5420f5fb92a37e1280d1fc9e5f4 -1           6.112790e-05
                                  0           9.403450e-01
                                  1           3.242947e-03
                                  2           5.628984e-02
ffaeae55d4dbf29058f04e7a6a764f02 -2           1.297567e-14
                                 -1           6.038120e-17
                                  0           5.831536e-15
                                  1           1.000000e+00
                                  2           5.212391e-15
ffc5289194413ec68c3f7adc8121d69b -2           4.921406e-12
                                 -1           4.921406e-12
                                  0           9.999998e-01
                                  1           2.722970e-09
                                  2           2.700765e-10
                                  3           2.171624e-07
ffd097949a4a238296a7deadfb376cc0 -1           1.464336e-04
                                  0           6.421584e-01
                                  1           3.155735e-01
                                  2           2.273007e-02
                                  3           1.939160e-02
ffdd5ec2a90e355cf40525eac1a6fd34 -2           2.210245e-06
                                 -1           1.063067e-09
                                  0           9.903741e-01
                                  1           8.184482e-03
                                  2           8.063902e-11
                                  3           1.439234e-03
                                  4           3.165577e-11
                                  5           5.756357e-10
                                  6           8.112334e-11
                                  7           3.521230e-11

[70906 rows x 1 columns]

In [278]:
y3 = y_pr2.merge(target_rank.dist_home.to_frame(), left_index=True, right_index=True)

In [280]:
def f(x):
    return x

In [281]:
y3.apply(f, axis=1)

y  dist_home
customer_id                      clust_label                         
0001f322716470bf9bfc1708f06f00fc -2           1.928413e-14        0.0
                                  0           1.038394e-04        1.0
                                  1           2.615784e-14        0.0
                                  2           9.998962e-01        0.0
0007297d86e14bd68bd87b1dbdefe302 -2           8.666884e-19        0.0
                                 -1           4.889978e-30        0.0
                                  0           9.122089e-27        0.0
                                  1           1.822909e-04        1.0
                                  2           8.393982e-28        0.0
                                  3           7.892106e-25        0.0
                                  4           9.998177e-01        0.0
                                  5           2.392669e-28        0.0
                                  6           1.355341e-27        0.0
                                  7           4.156258e-28        0.0
                                  8           2.790112e-26        0.0
000b709c6c6fb1e8efcfd95e57c2a9de -2           9.949515e-01        0.0
                                 -1           1.900149e-03        0.0
                                  0           1.068146e-04        0.0
                                  1           4.476657e-04        0.0
                                  2           7.636806e-04        1.0
                                  3           4.979079e-04        0.0
                                  4           1.249823e-04        0.0
                                  5           4.625071e-05        0.0
                                  6           2.599049e-04        0.0
                                  7           9.011863e-04        0.0
0027a7618d97cc9fbda55fac457eaeb7 -1           2.320674e-04        0.0
                                  0           9.189594e-01        0.0
                                  1           2.833914e-02        0.0
                                  2           5.246943e-02        1.0
002b9f6e118c54f1292e03d1a04d516e -2           4.759171e-26        0.0
...                                                    ...        ...
ff92d5420f5fb92a37e1280d1fc9e5f4 -1           6.112790e-05        0.0
                                  0           9.403450e-01        1.0
                                  1           3.242947e-03        0.0
                                  2           5.628984e-02        0.0
ffaeae55d4dbf29058f04e7a6a764f02 -2           1.297567e-14        0.0
                                 -1           6.038120e-17        1.0
                                  0           5.831536e-15        0.0
                                  1           1.000000e+00        0.0
                                  2           5.212391e-15        0.0
ffc5289194413ec68c3f7adc8121d69b -2           4.921406e-12        0.0
                                 -1           4.921406e-12        0.0
                                  0           9.999998e-01        0.0
                                  1           2.722970e-09        1.0
                                  2           2.700765e-10        0.0
                                  3           2.171624e-07        0.0
ffd097949a4a238296a7deadfb376cc0 -1           1.464336e-04        0.0
                                  0           6.421584e-01        0.0
                                  1           3.155735e-01        1.0
                                  2           2.273007e-02        0.0
                                  3           1.939160e-02        0.0
ffdd5ec2a90e355cf40525eac1a6fd34 -2           2.210245e-06        0.0
                                 -1           1.063067e-09        0.0
                                  0           9.903741e-01        0.0
                                  1           8.184482e-03        1.0
                                  2           8.063902e-11        0

In [117]:
coeffs = np.random.rand(sz, 1)
coeffs.shape

(252, 1)

In [180]:
d = train_features.loc[[test_user],:]
s1 = np.matmul(d,coeffs)
s1[:,0]

array([ 24.43966138, 136.04210817,  25.52307444, 179.55342921])

In [169]:
softmax([s1-s1.max()])

array([[[4.31465985e-68],
        [1.26845005e-19],
        [1.27487286e-67],
        [1.00000000e+00]]])

In [170]:
s1-s1.max()

array([[-155.11376783],
       [ -43.51132105],
       [-154.03035477],
       [   0.        ]])

In [159]:
def softmax2(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div

In [164]:
x2 = np.array([[1.0, 2, 3, 6],  # sample 1
               [2, 4.0, 5, 6],  # sample 2
               [1, 2, 3, 6]]) # sample 1 again(!)

In [165]:
softmax(x2)

array([[0.00626879, 0.01704033, 0.04632042, 0.93037047],
       [0.01203764, 0.08894682, 0.24178252, 0.65723302],
       [0.00626879, 0.01704033, 0.04632042, 0.93037047]])

In [73]:
ranker = FedRank()

In [80]:
features = ['dist_home', 'dist_work']
y = ranker.transform(p, features)
y.head(10)

,dist_home_rank,dist_work_rank
1,3,3
2,1,2
3,2,1
5,1,2
6,2,2
7,4,2
8,3,2
11,2,2
12,1,1
15,3,2


In [83]:
features = ['amount_mean', 'amount_count', 'amount_std']
X = ranker.transform(p, features)
X.head()

,amount_mean_rank,amount_count_rank,amount_std_rank
1,2,2,2
2,3,1,3
3,1,2,1
5,4,4,4
6,3,1,3


In [84]:
y.head()

,dist_home_rank,dist_work_rank
1,3,3
2,1,2
3,2,1
5,1,2
6,2,2


### Обучение и тестирование

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Подбор гиперпараметров для DBSCAN

In [ ]:
eps_list = np.logspace(-2, -1, 5)
min_samples_list = [5, 10, 15, 20, 25, 30]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

In [ ]:
eps_list = np.logspace(-3, -1, 10)
min_samples_list = [1, 2, 3, 4, 5]

res = {}

for min_samples in min_samples_list:
    for eps in eps_list: 
        res_df = get_dbscan_score(customer_gr, {'leaf_size':5, 'eps': eps, 'min_samples': min_samples})
        score = res_df.query('dist < @r2').shape[0]
        print(f'{eps}, {min_samples}, {score}')
        res.update({(eps, min_samples,):score})

Наилучшие результаты получены при eps = 0.036, min_samples = 1

In [ ]:
train_pr = get_dbscan_score(train_gr, best_dbscan_params)

In [ ]:
train_pr.head()

In [ ]:
train_pr.clust_amount.hist(bins = 100)
plt.xlim([0, 20])

Теперь оценим реальный скор, который я могу выбить

In [ ]:
0.5* train_pr.query('dist <= @r2').shape[0]/train_pr.shape[0]

На борде скор 0.206375

In [ ]:
test_pr = get_dbscan_score(test_gr, best_dbscan_params, calc_dist= False)
test_pr.head()

In [ ]:
submit = test_pr.loc[:,['home_post_lat', 'home_post_lon', 'best_post_lat',  'best_post_lon']]
submit.head()

In [ ]:
submit.columns = ['best_post_lat', 'best_post_lon',  'home_post_lat', 'home_post_lon', 'dist', 'clust_size']

In [ ]:
submit.to_csv('1.csv')

In [201]:
import pandas as pd
import numpy as np

columns = ['col{}'.format(i) for i in range(36)]
x = pd.DataFrame(np.random.random((1062, 36)), columns=columns)
y = pd.DataFrame(np.random.random((36, 36)))

print(np.dot(x, y).shape)
# (1062, 36)

(1062, 36)
